<a href="https://colab.research.google.com/github/neelsoumya/intro_to_LMMs/blob/main/smolagents_websearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Coding agents using the `smolagents` library. This notebook shows how to use web search in agents.

Acknowledgements:

https://learn.deeplearning.ai/courses/building-code-agents-with-hugging-face-smolagents/lesson/tv10w/introduction-to-code-agents


Install packages

In [1]:
!pip install 'smolagents[e2b]'
#!pip install git+https://github.com/huggingface/smolagents.git
!pip install e2b-code-interpreter

In [2]:
from smolagents.local_python_executor import LocalPythonExecutor
from smolagents import CodeAgent, HfApiModel, Tool
import os
from dotenv import load_dotenv
from google.colab import drive

In [3]:
custom_executor = LocalPythonExecutor(["numpy"])

Mount Google Drive

In [4]:
mount = "/content/gdrive"
drive.mount(mount)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Switch to drive on Google Drive you want

In [5]:
drive_root = mount + "/My Drive/intro_to_LMMs"
%cd $drive_root

/content/gdrive/My Drive/intro_to_LMMs


Create EBI KEY

https://e2b.dev/sign-up

https://e2b.dev/dashboard?tab=keys




In [6]:
E2B_API_KEY = os.getenv("E2B_API_KEY")
E2B_API_KEY = "YOUR_E2B_API_KEY"


Create Web search API

In [11]:
model = HfApiModel()

class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def __init__(self, max_output_length: int = 40000):
        super().__init__()
        self.max_output_length = max_output_length

    def forward(self, url: str) -> str:
        try:
            import re
            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException
            from smolagents.utils import truncate_content
            # Assign to self for later use
            self.re = re
            self.requests = requests
            self.markdownify = markdownify
            self.RequestException = RequestException
            self.truncate_content = truncate_content

        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, self.max_output_length)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"


Create agent

In [21]:
agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=model,
    executor_type="e2b",
    executor_kwargs={"api_key": E2B_API_KEY},#, "dependencies":["smolagents","requests","markdownify","re"]},
    additional_authorized_imports=["pandas", "matplotlib", "seaborn", "smolagents"],
    max_steps = 5
)

Initializing executor, hold on...

Requirement already satisfied: pandas in /usr/local/lib/python3.10/site-packages (1.5.3)
Requirement already satisfied: matplotlib in /usr/local/lib/python3.10/site-packages (3.9.2)
Requirement already satisfied: seaborn in /usr/local/lib/python3.10/site-packages (0.13.2)

Collecting smolagents

  Downloading smolagents-1.14.0-py3-none-any.whl (114 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/114.5 kB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m114.5/114.5 kB[0m [31m4.9 MB/s[0m eta [36m0:00:00[0m
[?25hRequirement already satisfied: numpy>=1.21.0 in /usr/local/lib/python3.10/site-packages (from pandas) (1.26.4)
Requirement already satisfied: python-dateutil>=2.8.1 in /usr/local/lib/python3.10/site-packages (from pandas) 
(2.9.0.post0)
Requirement already satisfied: pytz>=2020.1 in /usr/local/lib/python3.10/site-packages (from pandas) (2024.1)

Requirement already satisfied: cycler>=0.10 in /usr/local/lib/python3.10/site-packages (from matplotlib) (0.12.1)
Requirement already satisfied: pillow>=8 in /usr/local/lib/python3.10/site-packages (from matplotlib) (10.3.0)
Requirement already satisfied: pyparsing>=2.3.1 in /usr/local/lib/python3.10/site-packages (from matplotlib) 
(3.2.3)
Requirement already satisfied: packaging>=20.0 in /usr/local/lib/python3.10/site-packages (from matplotlib) (25.0)
Requirement already satisfied: kiwisolver>=1.3.1 in /usr/local/lib/python3.10/site-packages (from matplotlib) 
(1.4.8)
Requirement already satisfied: contourpy>=1.0.1 in /usr/local/lib/python3.10/site-packages (from matplotlib) 
(1.3.2)
Requirement already satisfied: fonttools>=4.22.0 in /usr/local/lib/python3.10/site-packages (from matplotlib) 
(4.57.0)

Collecting huggingface-hub>=0.28.0
  Downloading huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/481.4 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m481.4/481.4 kB[0m [31m15.0 MB/s[0m eta [36m0:00:00[0m
[?25h
Collecting python-dotenv

  Downloading python_dotenv-1.1.0-py3-none-any.whl (20 kB)

Collecting rich>=13.9.4
  Downloading rich-14.0.0-py3-none-any.whl (243 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/243.2 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m243.2/243.2 kB[0m [31m18.5 MB/s[0m eta [36m0:00:00[0m
[?25h
Collecting pillow>=8
  Downloading pillow-11.2.1-cp310-cp310-manylinux_2_28_x86_64.whl (4.6 MB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/4.6 MB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [91m━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.6/4.6 MB[0m [31m18.5 MB/s[0m eta
[36m0:00:01[0m[2K     [91m━━━━━━━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m1.3/4.6 MB[0m [31m19.1
MB/s[0m eta [36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━━━━━━━━━[0m [32m1.8/4.6 MB[0m [31m17.9 MB/s[0m eta
[36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━━━━━[0m [32m2.3/4.6 MB[0m [31m16.5 MB/s[0m eta
[36m0:00:01[0m[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━[0m[90m╺[0m[90m━━━━━━━━━━━━━━━[0m [32m2.8/4.6 MB[0m [31m15.7
MB/s[0m eta [36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m[90m━━━━━━━━━━━━[0m [32m3.2/4.6 MB[0m [31m15.0 MB/s[0m eta
[36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m[90m━━━━━━━━[0m [32m3.6/4.6 MB[0m [31m14.7 MB/s[0m eta
[36m0:00:01[0m[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m[90m━━━━[0m [32m4.1/4.6 MB[0m [31m14.6
MB/s[0m eta [36m0:00:01[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[91m╸[0m [32m4.6/4.6 MB[0m [31m14.4 MB/s[0m eta 
[36m0:00:01[0m
[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m4.6/4.6 

E2B is running

Call agent with task

In [22]:
output = agent.run("Give me output of the top github repos from the HuggingFace organization")

print(output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me output of the top github repos from the HuggingFace organization                                        │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

Requirement already satisfied: requests in /usr/local/lib/python3.10/site-packages (2.32.3)
Requirement already satisfied: markdownify in /usr/local/lib/python3.10/site-packages (1.1.0)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.10/site-packages (from requests) (3.10)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.10/site-packages (from requests) 
(1.26.19)
Requirement already satisfied: charset-normalizer<4,>=2 in /usr/local/lib/python3.10/site-packages (from requests) 
(3.4.2)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.10/site-packages (from requests) 
(2025.4.26)
Requirement already satisfied: beautifulsoup4<5,>=4.9 in /usr/local/lib/python3.10/site-packages (from markdownify)
(4.12.3)
Requirement already satisfied: six<2,>=1.15 in /usr/local/lib/python3.10/site-packages (from markdownify) (1.17.0)
Requirement already satisfied: soupsieve>1.2 in /usr/local/lib/python3.10/site-packages (from 
beautifulsoup4<5,>=4.9->markdownify) (2.7)

[33mWARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the 
system package manager. It is recommended to use a virtual environment instead: 
https://pip.pypa.io/warnings/venv[0m[33m
[0m
[1m[[0m[34;49mnotice[0m[1;39;49m][0m[39;49m A new release of pip is available: [0m[31;49m23.0.1[0m[39;49m ->
[0m[32;49m25.1.1[0m
[1m[[0m[34;49mnotice[0m[1;39;49m][0m[39;49m To update, run: [0m[32;49mpip install --upgrade pip[0m

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
401 Client Error: Unauthorized for url: 
https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: 
Root=1-6818c8e1-062bed097524dd2b55663d7b;590f39fb-492b-4120-b461-53ba46d9593c)

Invalid username or password.

[Step 1: Duration 0.05 seconds]

AgentGenerationError: Error in generating model output:
401 Client Error: Unauthorized for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-6818c8e1-062bed097524dd2b55663d7b;590f39fb-492b-4120-b461-53ba46d9593c)

Invalid username or password.